In [2]:
import os
os.chdir('../')

# 使用MMRazor对ResNet34进行剪枝

1. 回顾MMCLs
2. 搜索最优剪枝结构
3. 剪枝

## 回顾MMCls

In [3]:
# Prepare config path
work_dir='./prune_example/'
config_path=f"{work_dir}/configs/"
! mkdir -p $config_path

pretrained_path='https://download.openmmlab.com/mmclassification/v0/resnet/resnet34_8xb32_in1k_20210831-f257d4e6.pth'

我们直接使用mmcls的配置文件。（这里使用了mmengine的跨库调用功能，请参考[MMEngine文档](https://mmengine.readthedocs.io/zh_CN/latest/tutorials/config.html#id11)）

In [4]:
! echo "_base_ = ['mmcls::resnet/resnet34_8xb32_in1k.py']" > $config_path/pretrain.py
! cat $config_path/pretrain.py

# Run config
# ! python ./tools/train.py $config_path/pretrain.py

_base_ = ['mmcls::resnet/resnet34_8xb32_in1k.py']


In [5]:
! timeout 10 python ./tools/train.py $config_path/pretrain.py

/root/miniconda3/envs/lab2t12/lib/python3.8/site-packages/mmengine/config/utils.py:51: UserWarning: There is not `Config` define in {'Name': 'convnext-base_3rdparty_in21k', 'Metadata': {'Training Data': 'ImageNet-21k', 'FLOPs': 15359124480, 'Parameters': 88591464}, 'In Collection': 'ConvNeXt', 'Results': None, 'Weights': 'https://download.openmmlab.com/mmclassification/v0/convnext/convnext-base_3rdparty_in21k_20220124-13b83eec.pth', 'Converted From': {'Weights': 'https://dl.fbaipublicfiles.com/convnext/convnext_base_22k_224.pth', 'Code': 'https://github.com/facebookresearch/ConvNeXt'}}
  warnings.warn(f'There is not `Config` define in {model_cfg}')
/root/miniconda3/envs/lab2t12/lib/python3.8/site-packages/mmengine/config/utils.py:51: UserWarning: There is not `Config` define in {'Name': 'convnext-large_3rdparty_in21k', 'Metadata': {'Training Data': 'ImageNet-21k', 'FLOPs': 34368026112, 'Parameters': 197767336}, 'In Collection': 'ConvNeXt', 'Results': None, 'Weights': 'https://download.

## 准备剪枝结构搜索config
1. 生成搜索config
2. 运行搜索config
3. 获得搜索出的最优结构

### 1.生成搜索config
我们提供了一个工具可以帮助我们一条命令行生成搜索算法config。该工具需要我们提供两个参数：config路径与checkpoint路径

In [6]:
! python tools/get_search_config.py -h

usage: get_search_config.py [-h] [--checkpoint CHECKPOINT]
                            [--flops-min FLOPS_MIN] [--flops-max FLOPS_MAX]
                            [-o O]
                            config

Get the config to search the pruning structure of a model

positional arguments:
  config                config of the model

optional arguments:
  -h, --help            show this help message and exit
  --checkpoint CHECKPOINT
                        checkpoint path of the model
  --flops-min FLOPS_MIN
                        minimal flops
  --flops-max FLOPS_MAX
                        maximal flops
  -o O                  output path to store the search config.


In [7]:
! python tools/get_search_config.py  $config_path/pretrain.py --checkpoint $pretrained_path -o $config_path/search.py   &> /dev/null 
! cat $config_path/search.py 

model = dict(
    _scope_='mmrazor',
    type='SearchWrapper',
    architecture=dict(
        type='ImageClassifier',
        backbone=dict(
            type='ResNet',
            depth=34,
            num_stages=4,
            out_indices=(3, ),
            style='pytorch'),
        neck=dict(type='GlobalAveragePooling'),
        head=dict(
            type='LinearClsHead',
            num_classes=1000,
            in_channels=512,
            loss=dict(type='CrossEntropyLoss', loss_weight=1.0),
            topk=(1, 5)),
        _scope_='mmcls',
        data_preprocessor=dict(
            mean=[123.675, 116.28, 103.53],
            std=[58.395, 57.12, 57.375],
            to_rgb=True),
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://download.openmmlab.com/mmclassification/v0/resnet/resnet34_8xb32_in1k_20210831-f257d4e6.pth'
        )),
    mutator_cfg=dict(
        type='ChannelMutator',
        channel_unit_cfg=dict(
            type

对于生成的config，我们可以进一步进行修改，以匹配自己的需求。其中"model"和“train_cfg”是最重要的两个字段。

### 2. 运行搜索config

In [8]:
! timeout 60 python ./tools/train.py $config_path/search.py  --work-dir $work_dir/search/

11/24 16:34:42 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.13 (default, Mar 28 2022, 11:38:47) [GCC 7.5.0]
    CUDA available: True
    numpy_random_seed: 462324642
    GPU 0: NVIDIA GeForce RTX 3060
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.3, V11.3.109
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
    PyTorch: 1.12.1+cu113
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.3
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,cod

### 3. 获取搜索结果

搜索结果存放在work_dir目录下面，我们可以通过以下命令查看我们最终的搜索结果。搜索结果用一个字典表示，字典的key是每个可搜索通道节点的名字（name），value则是该通道节点保留的通道比例。

In [9]:
!  cat $work_dir/search/best_fix_subnet.json

cat: ./prune_example//search/best_fix_subnet.json: No such file or directory


## 准备剪枝Config

我们提供了一键生成剪枝config的工具get_prune_config.py


In [10]:
! python ./tools/get_prune_config.py -h

usage: get_prune_config.py [-h] [--checkpoint CHECKPOINT] [--subnet SUBNET]
                           [-o O]
                           config

Get the config to prune a model.

positional arguments:
  config                config of the model

optional arguments:
  -h, --help            show this help message and exit
  --checkpoint CHECKPOINT
                        checkpoint path of the model
  --subnet SUBNET       pruning structure for the model
  -o O                  output path to store the pruning config.


In [14]:
! python ./tools/get_prune_config.py $config_path/pretrain.py --checkpoint $pretrained_path --subnet $work_dir/search/best_fix_subnet.json -o $config_path/prune.py  &> /dev/null
! cat $config_path/prune.py

model = dict(
    _scope_='mmrazor',
    type='ItePruneAlgorithm',
    architecture=dict(
        type='ImageClassifier',
        backbone=dict(
            type='ResNet',
            depth=34,
            num_stages=4,
            out_indices=(3, ),
            style='pytorch'),
        neck=dict(type='GlobalAveragePooling'),
        head=dict(
            type='LinearClsHead',
            num_classes=1000,
            in_channels=512,
            loss=dict(type='CrossEntropyLoss', loss_weight=1.0),
            topk=(1, 5)),
        _scope_='mmcls',
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://download.openmmlab.com/mmclassification/v0/resnet/resnet34_8xb32_in1k_20210831-f257d4e6.pth'
        ),
        data_preprocessor=dict(
            mean=[123.675, 116.28, 103.53],
            std=[58.395, 57.12, 57.375],
            to_rgb=True)),
    target_pruning_ratio=dict({
        'backbone.conv1_(0, 64)_64': 1.0,
        'backbone.laye

## 剪枝

In [15]:
# run your prune config
! timeout 10 python ./tools/train.py $config_path/prune.py --work-dir $work_dir/prune

11/24 16:36:46 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.13 (default, Mar 28 2022, 11:38:47) [GCC 7.5.0]
    CUDA available: True
    numpy_random_seed: 1781704939
    GPU 0: NVIDIA GeForce RTX 3060
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.3, V11.3.109
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
    PyTorch: 1.12.1+cu113
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.3
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,co

In [13]:
# ! rm -r $work_dir